In [1]:

%pprint

Pretty printing has been turned OFF



---
# Load needed libraries and functions

In [2]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [3]:

# Get the Neo4j driver
from storage import Storage
s = Storage()

from ha_utils import HeaderAnalysis
ha = HeaderAnalysis(s=s, verbose=False)

from scrape_utils import WebScrapingUtilities
wsu = WebScrapingUtilities(s=s)
uri = wsu.secrets_json['neo4j']['connect_url']
user =  wsu.secrets_json['neo4j']['username']
password = wsu.secrets_json['neo4j']['password']

from cypher_utils import CypherUtilities
cu = CypherUtilities(uri=uri, user=user, password=password, driver=None, s=s, ha=ha)

In [4]:

from neo4j.exceptions import ServiceUnavailable

try:
    version_str = cu.driver.verify_connectivity()
    print(f'======== {version_str} ========')
    
    from hc_utils import HeaderCategories
    hc = HeaderCategories(cu=cu, verbose=False)
    
    from section_utils import SectionUtilities
    su = SectionUtilities(verbose=False)
    
    from lr_utils import LrUtilities
    lru = LrUtilities(ha=ha, cu=cu, hc=hc, verbose=False)
    
    from crf_utils import CrfUtilities
    crf = CrfUtilities(ha=ha, hc=hc, cu=cu, verbose=False)
    
    import warnings
    warnings.filterwarnings('ignore')
except ServiceUnavailable as e:
    # print(str(e).strip())
    raise ServiceUnavailable('You need to start Neo4j as a console')
except Exception as e:
    print(f'{e.__class__}: {str(e).strip()}')

======== Neo4j/4.4.7 ========


In [5]:

%matplotlib inline
import time
import humanize
import matplotlib.pyplot as plt
from datetime import datetime
import winsound

duration = 1000  # milliseconds
freq = 880  # Hz
width_inches = 18.0
height_inches = 3.0
bin_count = 12
print(f'Last run on {datetime.now()}')

Last run on 2023-01-17 10:50:37.428865


In [167]:

import re
import pandas as pd

sentence_regex = re.compile(r'[;•➢\*]|\.(?!\w)')
html_regex = re.compile(r'</?[^>]+>')
def do_cypher_tx(tx, verbose=False):
    cypher_str = """MATCH (np:NavigableParents)
        WHERE
            np.is_header = 'True' AND
            np.is_minimum_qualification = 'True'
        RETURN np.navigable_parent;"""
    results_list = tx.run(query=cypher_str, parameters=None)

    return [dict(record.items())['np.navigable_parent'] for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx)
phrases_list = []
for child_str in row_objs_list:
    if bytearray(child_str[0], encoding='utf-8', errors='replace') == bytearray(b'-'):
        child_str = child_str[1:]
    child_str = html_regex.sub('', child_str.strip('•\xa0 '))
    phrases_list.append(child_str)
pd.DataFrame(phrases_list).value_counts().sort_values(ascending=False).head(20)

Qualifications              7
Experience:                 6
Requirements:               5
Required:                   4
Minimum Qualifications:     4
Qualifications:             4
Required Qualifications:    4
Basic Qualifications        3
Required Experience:        3
Skills Required:            3
Skills:                     3
Minimum Requirements:       3
Requirements                3
About You                   3
What You Need:              3
Required                    3
Required Skills:            3
Skills                      2
Required Skills             2
Skills and Experience:      2
dtype: int64

In [119]:

print(phrases_list[2], '-'.join(str(x) for x in bytearray(str(phrases_list[2]), encoding='utf-8', errors='replace')))
bytearray(phrases_list[2][0], encoding='utf-8', errors='replace') == bytearray(b'-')

- Python 45-32-80-121-116-104-111-110


True

In [156]:

def do_cypher_tx(tx, verbose=False):
    cypher_str = """MATCH (np:NavigableParents)
        WHERE
            (np.is_header = 'True') AND
            (np.navigable_parent CONTAINS ' we are looking for ') AND
            (NOT np.navigable_parent ENDS WITH ':') AND
            (NOT np.navigable_parent ENDS WITH ':</div>') AND
            (NOT np.navigable_parent ENDS WITH ', Including The Following Areas')
        SET np.is_header = 'False'
        RETURN np.navigable_parent;"""
    results_list = tx.run(query=cypher_str, parameters=None)

    return [dict(record.items())['np.navigable_parent'] for record in results_list]
with cu.driver.session() as session:
    row_objs_list = session.write_transaction(do_cypher_tx)
ihu.retrain_classifier(row_objs_list, ['True']*len(row_objs_list), verbose=True)
row_objs_list

['Currently, we are looking for talented resources for one of our listed clients. If interested please reply to me with your updated resume or feel free to reach out to me for more details at', '<b>What we are looking for </b>', '<p>In short, we are looking for a Difference Maker .</p>', "<p>If you are an engineer who's passionate about building innovative products that scale to tens of millions of pageviews a day and to help people find jobs globally, we are looking for you.</p>", '<b>For this position we are looking for an experienced data scientist with:</b>', '<p>In addition to the desire to learn, willingness to move fast and work hard, and enthusiasm for (or at least acceptance of) many Star Wars references, we are looking for candidates with the following:</p>', '<p>In short, we are looking for a “Difference Maker”.</p>', '<p>Our studio is pioneering new methods to blend AI and gameplay, and we are looking for a strong software engineer to drive first-of-its-kind gameplay experi

In [49]:

chr(45)

'-'